In [9]:
import numpy as np
import pandas as pd

"""

Only two agents will be spawned.
This will be a simple sim based on the
example I thought of when I was on my mission.
Only two agents exist in an entire universe with limited amount of
"starstuff".
Starstuff is used to make energy for the agent, to fashion into coinage, to create tools and materials
and to trade in commercial transactions.

Trump controls all of the starstuff and maintains full control over Chump for one simple reason: Trump possesses
the ability to convert starstuff into consumable energy which allows both of them to survive.

Chump works for Trump; performing several jobs in exchange for payment in starcoins.
Chump's jobs are:
    1. mine starstuff
    2. mint starcoins from starstuff
    3. construct usable machines and assemblies from starstuff (cars, homes, batteries, ... whatever)

"""



'\n\nOnly two agents will be spawned.\nThis will be a simple sim based on the\nexample I thought of when I was on my mission.\nOnly two agents exist in an entire universe with limited amount of\n"starstuff".\nStarstuff is used to make energy for the agent, to fashion into coinage, to create tools and materials\nand to trade in commercial transactions.\n\nTrump controls all of the starstuff and maintains full control over Chump for one simple reason: Trump possesses\nthe ability to convert starstuff into consumable energy which allows both of them to survive.\n\nChump works for Trump; performing several jobs in exchange for payment in starcoins.\nChump\'s jobs are:\n    1. mine starstuff\n    2. mint starcoins from starstuff\n    3. construct usable machines and assemblies from starstuff (cars, homes, batteries, ... whatever)\n\n'

In [10]:

# STARTING VARIABLES

# ORBITS_PER_SIMULATION   = 100 # years
# REVOLUTIONS_PER_ORBIT   = 365 # days
# ARCS_PER_REVOLUTION     = 24 # hours
ORBITS_PER_SIMULATION   = 1 # years
REVOLUTIONS_PER_ORBIT   = 20 # days
ARCS_PER_REVOLUTION     = 24 # hours

miniverse_starstuff_total   = 2985984 # 2985984 because it's a cubic number and that's how I imagined it ... deal with it

# agents' starting values
starting_values = dict()
starting_values['energy'] = 240
starting_values['food'] = 240

# stuff to food conversion ratio
starstuff_2_starfood        = 40 # 1 starstuff makes 40 starfoods


In [11]:

class Action:

    action_name                 = '_NULL_'
    hourly_energy_expenditure   = 0 # how much energy is necessary per single hour of this action



In [12]:

# keep the list minimal
# I think the following list should do:
#   1. house
#   2. car
#   3. THE utility (phone, electric, water, sewer, internet ... all wrapped up into one)
#   4. fringe possessions (clothes, furniture, electronics, etc. all wrapped up into one)
class Possession:

    # what is necessary to create this item
    creation_values = dict()
    creation_values['starstuff']    = 0 # how much star stuff was used to make it
    creation_values['energy']       = 0 # how much energy expended to make it

    # monetary values
    starcoin_values = dict()
    starcoin_values['new']      = 0
    starcoin_values['current']  = 0

    # as close to "inherent" value as this can get
    deterioration_metrics = dict()
    # how strong it is right after manufacture?
    # Below 75% of it's starting quality will render it unusable
    deterioration_metrics['starting_quality']        = 0 # ranges from 0 to 100 percent
    deterioration_metrics['current_quality']         = 0 # ranges from 0 to 100 percent
    deterioration_metrics['temporal_depreciation']   = 0 # linear wear and tear per hour
    deterioration_metrics['wear_and_tear']           = 0 # accelerated wear and tear when in use

    # this catalogues how much more efficient some particular action is
    # when this particular possession is in use
    # (ie: what this possession was designed to help with)
    purpose = dict()
    purpose['action'] = ''       # which agent action was this possession manufactured to help with?
    purpose['effect'] = ''



In [13]:

class S3DS_Agent:

    energy = 0

    starstuff = 0
    starcoins = 0
    starfood  = 0

    possessions = list()

    def __init__(self, starting_vals:dict):
        self.energy     = starting_vals['energy']
        self.starfood   = starting_vals['food']

    def move(self, hour):

        energy_expended = 0

        # assign a schedule for both agents (we can keep it simple and give them both the same schedule)
        # NOTE: maximum daily energy expenditure, IF the schedule is perfectly followed
        #       AND we calculate for the maximum possible expenditure for any one activity state
        #       we get: 131 energy units (as is displayed below)

        schedule = {0   :'sleep',       # 2
                    1   :'sleep',       # 2
                    2   :'sleep',       # 2
                    3   :'sleep',       # 2
                    4   :'sleep',       # 2
                    5   :'sleep',       # 2
                    6   :'daily_prep',  # 4
                    7   :'eat',         # 3
                    8   :'travel',      # 3
                    9   :'work',        # 5 to 10
                    10  :'work',        # 5 to 10
                    11  :'work',        # 5 to 10
                    12  :'work',        # 5 to 10
                    13  :'eat',         # 3
                    14  :'work',        # 5 to 10
                    15  :'work',        # 5 to 10
                    16  :'work',        # 5 to 10
                    17  :'work',        # 5 to 10
                    18  :'travel',      # 3
                    19  :'exercise',    # 8 to 10
                    20  :'eat',         # 3
                    21  :'shop',        # 6
                    22  :'sleep',       # 2
                    23  :'sleep'}       # 2

        if schedule[hour] == 'sleep':
            self.sleep()
        if schedule[hour] == 'daily_prep':
            self.daily_prep()
        if schedule[hour] == 'eat':
            self.eat()
        if schedule[hour] == 'travel':
            self.travel()
        if schedule[hour] == 'work':
            self.work()
        if schedule[hour] == 'exercise':
            self.exercise()
        if schedule[hour] == 'shop':
            self.shop()

        # for i in self.possessions:
        #     energy_expended += i.required_energy_use

    def sleep(self):
        # energy used for sleeping
        self.energy -= 2

    def daily_prep(self):
        # energy required for prepping
        self.energy -= 4

    def eat(self):
        # energy require for eating
        self.energy -= 3

        # determine how much food is needed
        # max daily expenditure (calculated above, at schedule) is 131
        mde = 131
        if self.energy < (mde//2):
            # prepare a portion of food
            self.starfood -= 1
            # energy gained from eating
            self.energy += starstuff_2_starfood

    def travel(self):
        self.energy -= 3

    def exercise(self):
        self.energy -= 10

    def shop(self):
        self.energy -= 6



In [14]:

class Trump_Agent(S3DS_Agent):

    def __init__(self, starting_vals):
        super().__init__(starting_vals)

    def work(self, req_for_job=10):
        self.energy -= req_for_job
        # decide what needs to be done next (inventory-wise)

        # then order Chump to handle it



In [15]:
class Chump_Agent(S3DS_Agent):

    def __init__(self, starting_vals):
        super().__init__(starting_vals)
        pass

    def work(self, req_for_job=10):
        self.energy -= req_for_job
        # take Trumps orders and complete them



In [16]:

# PRE-LOOP PREPARATIONS

# instantiate agents and prepare the "board"
trump = Trump_Agent(starting_values)
chump = Chump_Agent(starting_values)



In [17]:

# GAME LOOP
# orbits    -> revolutions  -> arcs
# years     -> days         -> hours
for standard_orbit in range(ORBITS_PER_SIMULATION):
    for standard_revolution in range(REVOLUTIONS_PER_ORBIT):
        for standard_arc in range(ARCS_PER_REVOLUTION):

            trump.move(standard_arc)
            chump.move(standard_arc)